In [ ]:
!pip install gym
!pip install torch
!pip install torchvision
!pip install atari_py
!pip install ale-py
!pip install gym[atari]
!pip install gym[accept-rom-license]
!pip install tianshou

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 540.6/540.6 kB 1.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for atari_py: filename=atari_py-0.2.9-cp310-cp310-linux_x86_64.whl size=2871799 sha256=04677bcf2a27e666ab1300d8d5aded08dfb69c3ec25d77f3ab2cf4bf74a18eee
  Stored in directory: /root/.cache/pip/wheels/75/6f/04/1f3bf5255580101e16ff487564354dddcdd23ec3b43b775b7a
Successfully built atari_py
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 25.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 24.8 MB/s eta 0:00:00
  Attempting uninstall: ale-py
    Found existing installation: ale-py 0.10.1
    Uninstalling ale-py-0.10.1:
      Successfully uninstalled ale-py-0.10.1
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 434.7/434.7 kB 8.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for AutoROM.accept-rom-license: 

In [ ]:
!pip install stable-baselines3[extra]


  Using cached ale_py-0.10.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (7.6 kB)
Using cached ale_py-0.10.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (2.1 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 4.8 MB/s eta 0:00:00
  Attempting uninstall: ale-py
    Found existing installation: ale-py 0.7.5
    Uninstalling ale-py-0.7.5:
      Successfully uninstalled ale-py-0.7.5


In [ ]:
import ale_py
import gymnasium as gym
from stable_baselines3 import DQN
from stable_baselines3.common.atari_wrappers import FireResetEnv
import numpy as np
from collections import deque

def _parse_reset_result(reset_result):
    contains_info = (
        isinstance(reset_result, tuple)
        and len(reset_result) == 2
        and isinstance(reset_result[1], dict)
    )
    if contains_info:
        return reset_result[0], reset_result[1], contains_info
    return reset_result, {}, contains_info


class FrameStack(gym.Wrapper):
    """Stack n_frames last frames.

    :param gym.Env env: the environment to wrap.
    :param int n_frames: the number of frames to stack.
    """

    def __init__(self, env, n_frames):
        super().__init__(env)
        self.n_frames = n_frames
        self.frames = deque([], maxlen=n_frames)
        shape = (n_frames, *env.observation_space.shape)
        self.observation_space = gym.spaces.Box(
            low=np.min(env.observation_space.low),
            high=np.max(env.observation_space.high),
            shape=shape,
            dtype=env.observation_space.dtype,
        )

    def reset(self, **kwargs):
        obs, info, return_info = _parse_reset_result(self.env.reset(**kwargs))
        for _ in range(self.n_frames):
            self.frames.append(obs)
        return (self._get_ob(), info) if return_info else self._get_ob()

    def step(self, action):
        step_result = self.env.step(action)
        if len(step_result) == 4:
            obs, reward, done, info = step_result
            new_step_api = False
        else:
            obs, reward, term, trunc, info = step_result
            new_step_api = True
        self.frames.append(obs)
        if new_step_api:
            return self._get_ob(), reward, term, trunc, info
        return self._get_ob(), reward, done, info

    def _get_ob(self):
        # the original wrapper use `LazyFrames` but since we use np buffer,
        # it has no effect
        return np.stack(self.frames, axis=0)

In [ ]:
def make_env(env_name, **kwargs):
    env = gym.make(env_name, **kwargs)
    env = FireResetEnv(env)
    env = gym.wrappers.AtariPreprocessing(
        env           = env,
        noop_max      = 0,
        frame_skip    = 4,
        screen_size   = 84,
        grayscale_obs = True,
        scale_obs     = True,
    )
    # Don't use gym.wrappers.FrameStack with Tianshou
    env = FrameStack(env, n_frames=4)
    return env


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
import torch
import torch.nn as nn
class DQN(nn.Module):

    def __init__(self, input_shape, n_actions, *args, **kwargs) -> None:
        super().__init__(*args, **kwargs)

        self.conv = nn.Sequential(
            nn.Conv2d(input_shape[0], 32, kernel_size=8, stride=4),
            nn.ReLU(),
            nn.Conv2d(32, 64, kernel_size=8, stride=2),
            nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size=3, stride=1),
            nn.ReLU(),
        )

        conv_out_size = self.get_conv_out_size(input_shape)

        # Or: LazyLinear
        self.fc = nn.Sequential(
            nn.Flatten(),
            nn.Linear(conv_out_size, 512),
            nn.ReLU(),
            nn.Linear(512, n_actions)
        )

    def get_conv_out_size(self, image_dim):
        return np.prod(self.conv(torch.rand(*image_dim)).data.shape)

    def forward(self, inp, state=None, **kwargs):
        inp = torch.tensor(inp).cuda()
        return self.fc(self.conv(inp)), state

In [ ]:
from tianshou.env import DummyVectorEnv, ShmemVectorEnv
from tianshou.data import Collector, PrioritizedVectorReplayBuffer
from tianshou.policy import DQNPolicy
from tianshou.trainer import offpolicy_trainer

#ENV_NAME = "PongNoFrameskip-v4"
ENV_NAME = "BreakoutNoFrameskip-v4"

# General hyperparams
GAMMA = .99
BATCH_SIZE = 64
LEARNING_RATE = 0.0001
SYNC_TARGET_FRAMES = 500
NUM_STACK = 4
STEP_PER_EPOCH = 100_000
STEP_PER_COLLECT = 10
UPDATE_PER_STEP = 0.1
MAX_EPOCH = 100
EPISODE_PER_STEP = 1

# Buffer
BUFFER_SIZE = 100_000
ALPHA = 0.5
BETA_START = 0.4
BETA_FINAL = 1.
BETA_ANNEAL = 5_000_000

# Epsilon for exploration.
EPS_START = 1.
EPS_FINAL = 0.01
EPSILON_DECAY = 1e6

# N ENVS
N_TRAIN_ENVS = 10
N_TEST_ENVS = 1

In [ ]:
train_envs = \
  ShmemVectorEnv([lambda: make_env(ENV_NAME) for _ in range(N_TRAIN_ENVS)])
test_env = \
  ShmemVectorEnv([lambda: make_env(ENV_NAME) for _ in range(N_TEST_ENVS)])

In [ ]:
net = DQN(
    train_envs.observation_space[0].shape,
    train_envs.action_space[0].n
)

optim = torch.optim.Adam(net.parameters(), lr=0.0001)

In [ ]:
policy = DQNPolicy(
    model              = net,
    optim              = optim,
    estimation_step    = 1,
    target_update_freq = SYNC_TARGET_FRAMES,
    discount_factor    = GAMMA,
    is_double          = False
).cuda()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
buffer = PrioritizedVectorReplayBuffer(
    total_size         = BUFFER_SIZE,
    buffer_num         = len(train_envs),
    alpha              = ALPHA,
    beta               = BETA_START,
    stack_num          = NUM_STACK,
    ignore_obs_next    = True,
    save_only_last_obs = True
)

train_collector = Collector(
    policy = policy,
    env    = train_envs,
    buffer = buffer,
    exploration_noise = True
)

# For test_collector, we don't need buffer nor exploration (random actions)
test_collector = Collector(policy, test_env)

In [ ]:
from torch.utils.tensorboard import SummaryWriter
from tianshou.utils import TensorboardLogger
import os

log_path = os.path.join('logs', "dqn")
writer = SummaryWriter(log_path)
logger = TensorboardLogger(writer)

def train_fn(epoch, env_step):
    eps = max(EPS_FINAL, EPS_START - env_step / EPSILON_DECAY * (EPS_START - EPS_FINAL))
    policy.set_eps(eps)

    if env_step <= BETA_ANNEAL:
        beta = BETA_START - env_step / BETA_ANNEAL * (BETA_START- BETA_FINAL)
    else:
        beta = BETA_FINAL
    buffer.set_beta(beta)

    if env_step % 1000 == 0:
        logger.write("train/env_step", env_step, {"train/eps": eps})
        logger.write("train/env_step", env_step, {"train/beta": beta})

In [ ]:
def save_checkpoint_fn(epoch: int, env_step: int, gradient_step: int):
    if env_step % 1000 == 0:
        ckpt_path = os.path.join(log_path, f"breakout_model.pth")
        torch.save({"breakout_model": policy.state_dict()}, ckpt_path)

In [ ]:
!pip install tensorboard
!pip install jupyter-tensorboard


  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 29.4 MB/s eta 0:00:00
  Created wheel for jupyter-tensorboard: filename=jupyter_tensorboard-0.2.0-py2.py3-none-any.whl size=15247 sha256=c09fee12e42577dec91326686d5d2f26b233b24e975af70bcbd5d4534f1a3e5e
  Stored in directory: /root/.cache/pip/wheels/6a/e8/ee/64103cb21821f6a747a19b93a417b7e925857daab8ec5c7f50
Successfully built jupyter-tensorboard


In [ ]:
#%load_ext tensorboard
#%tensorboard --logdir logs/dqn

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
result = offpolicy_trainer(
    policy              = policy,
    train_collector     = train_collector,
    test_collector      = test_collector,
    max_epoch           = MAX_EPOCH,
    step_per_epoch      = STEP_PER_EPOCH,
    step_per_collect    = STEP_PER_COLLECT,
    update_per_step     = UPDATE_PER_STEP,
    episode_per_test    = EPISODE_PER_STEP,
    stop_fn             = lambda mean_rewards: mean_rewards >= 17,
    save_checkpoint_fn  = save_checkpoint_fn,
    batch_size          = BATCH_SIZE,
    train_fn            = train_fn,
    test_fn             = lambda epoch, env_step: policy.set_eps(0.0),
    logger              = logger,
    test_in_train       = False,
)


Epoch #1: 100001it [09:15, 180.17it/s, env_step=100000, len=196, loss=0.002, n/ep=0, n/st=10, rew=1.00]                            


Epoch #1: test_reward: 0.000000 ± 0.000000, best_reward: 0.000000 ± 0.000000 in #0


Epoch #2: 100001it [09:06, 183.11it/s, env_step=200000, len=209, loss=0.005, n/ep=1, n/st=10, rew=2.00]                            


Epoch #2: test_reward: 6.000000 ± 0.000000, best_reward: 6.000000 ± 0.000000 in #2


Epoch #3: 100001it [09:10, 181.63it/s, env_step=300000, len=232, loss=0.005, n/ep=0, n/st=10, rew=3.00]                            


...

Epoch #4: test_reward: 12.000000 ± 0.000000, best_reward: 12.000000 ± 0.000000 in #4


Epoch #5: 100001it [09:03, 183.96it/s, env_step=500000, len=495, loss=0.007, n/ep=0, n/st=10, rew=9.00]                            


Epoch #5: test_reward: 16.000000 ± 0.000000, best_reward: 16.000000 ± 0.000000 in #5


Epoch #6: 100001it [09:02, 184.48it/s, env_step=600000, len=242, loss=0.009, n/ep=0, n/st=10, rew=3.00]                            


Epoch #6: test_reward: 40.000000 ± 0.000000, best_reward: 40.000000 ± 0.000000 in #6


In [ ]:
!zip -r logs.zip /content/logs


updating: content/logs/ (stored 0%)
updating: content/logs/dqn/ (stored 0%)
updating: content/logs/dqn/events.out.tfevents.1733072497.717ee8830005.562.0 (deflated 68%)
updating: content/logs/dqn/pong_model.pth (deflated 7%)
  adding: content/logs/dqn/events.out.tfevents.1733082923.717ee8830005.562.1 (deflated 69%)
  adding: content/logs/dqn/breakout_model.pth (deflated 7%)


In [ ]:
from google.colab import files
files.download('/content/logs.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
def make_video(output_path='video/'):
    policy.eval()
    rec_env = DummyVectorEnv(
        [
            lambda: gym.wrappers.RecordVideo(
                make_env(ENV_NAME, render_mode="rgb_array", ),
                         #num_stack=NUM_STACK),
                video_folder=output_path
            )
        ]
    )
    rec_env.seed(7)
    collector = Collector(policy, rec_env, exploration_noise=False)
    collector.collect(n_episode=1)

make_video()


/usr/local/lib/python3.10/dist-packages/moviepy/config_defaults.py:1: DeprecationWarning: invalid escape sequence '\P'
  """


In [ ]:
!zip -r video.zip /content/video
from google.colab import files
files.download('/content/video.zip')

  adding: content/video/ (stored 0%)
  adding: content/video/rl-video-episode-0.mp4 (deflated 30%)
  adding: content/video/rl-video-episode-1.mp4 (deflated 24%)


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>